In [3]:
import yfinance as yf
import pandas as pd
import numpy as np

pfe = yf.Ticker("PFE")
pfe.info

{'address1': '66 Hudson Boulevard East',
 'city': 'New York',
 'state': 'NY',
 'zip': '10001-2192',
 'country': 'United States',
 'phone': '212 733 2323',
 'website': 'https://www.pfizer.com',
 'industry': 'Drug Manufacturers - General',
 'industryKey': 'drug-manufacturers-general',
 'industryDisp': 'Drug Manufacturers - General',
 'sector': 'Healthcare',
 'sectorKey': 'healthcare',
 'sectorDisp': 'Healthcare',
 'longBusinessSummary': "Pfizer Inc. discovers, develops, manufactures, markets, distributes, and sells biopharmaceutical products worldwide. It offers medicines and vaccines in various therapeutic areas, including cardiovascular metabolic, migraine, and women's health under the Eliquis, Nurtec ODT/Vydura, and the Premarin family brands; infectious diseases with unmet medical needs under the Prevnar family, Nimenrix, FSME/IMMUN-TicoVac, and Trumenba brands; and COVID-19 prevention and treatment, and potential future mRNA and antiviral products under the Comirnaty and Paxlovid br

In [4]:
tickers = ["PFE", "MSFT", "AAPL", "GOOG", "AMZN"]
import matplotlib.pyplot as plt
data = yf.download(tickers, start="2023-10-05", end="2023-10-10")
data.head(2)

[*********************100%%**********************]  5 of 5 completed


Adj Close                                                 \
                  AAPL        AMZN        GOOG        MSFT        PFE   
Date                                                                    
2023-10-05  174.910004  125.959999  135.990005  319.359985  33.470001   
2023-10-06  177.490005  127.959999  138.729996  327.260010  33.130001   

                 Close                                                 ...  \
                  AAPL        AMZN        GOOG        MSFT        PFE  ...   
Date                                                                   ...   
2023-10-05  174.910004  125.959999  135.990005  319.359985  33.470001  ...   
2023-10-06  177.490005  127.959999  138.729996  327.260010  33.130001  ...   

                  Open                                                 \
                  AAPL        AMZN        GOOG        MSFT        PFE   
Date                                                                    
2023-10-05  173.789993  126.709999  136.130005  319.089996  33.450001   
2023-10-06  173.800003  124.160004  134.940002  316.549988  33.520000   

              Volume                                          
                AAPL      AMZN      GOOG      MSFT       PFE  
Date                                                          
2023-10-05  48527900  39660600  15922900  16965600  21279400  
2023-10-06  57224100  46795900  20819300  25645500  23750600  

[2 rows x 30 columns]

In [5]:
amazon = yf.Ticker("AMZN")
hist = amazon.history(period="1mo")
hist['Date'] = hist.index
hist= hist.reset_index(drop=True)
hist.head(2)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Date
0,138.750000,143.619995,138.639999,143.100006,56764500,0.0,0.0,2023-09-11 00:00:00-04:00
1,142.320007,143.000000,140.610001,141.229996,42668500,0.0,0.0,2023-09-12 00:00:00-04:00


In [7]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

CODER_REDSHIFT_HOST = os.environ.get('CODER_REDSHIFT_HOST')
CODER_REDSHIFT_DB = os.environ.get('CODER_REDSHIFT_DB')
CODER_REDSHIFT_USER = os.environ.get('CODER_REDSHIFT_USER')
CODER_REDSHIFT_PASS = os.environ.get('CODER_REDSHIFT_PASS')
CODER_REDSHIFT_PORT = os.environ.get('CODER_REDSHIFT_PORT')

try:
    conn = psycopg2.connect(
        host=CODER_REDSHIFT_HOST,
        dbname=CODER_REDSHIFT_DB,
        user=CODER_REDSHIFT_USER,
        password=CODER_REDSHIFT_PASS,
        port=CODER_REDSHIFT_PORT,

    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


In [17]:
hist.dtypes


Open                                     float64
High                                     float64
Low                                      float64
Close                                    float64
Volume                                     int64
Dividends                                float64
Stock Splits                             float64
Date            datetime64[ns, America/New_York]
dtype: object

In [42]:
from psycopg2.extras import execute_values

def cargar_en_redshift(conn, table_name, dataframe):
    dtypes = dataframe.dtypes
    cols = list(dtypes.index)
    print(cols)
    tipos = list(dtypes.values)
    type_map = {
        'float64': 'FLOAT',
        'int64': 'INT',
        'datetime64[ns, America/New_York]': 'TIMESTAMP'
    }
    sql_dtypes = [type_map.get(str(dtype), 'VARCHAR(255)') for dtype in tipos]

    # Definir formato SQL VARIABLE TIPO_DATO
    column_defs = [f'"{name}" {data_type}' for name, data_type in zip(cols, sql_dtypes)]

    # Combine column definitions into the CREATE TABLE statement
    table_schema = f"""
        CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(column_defs)});
        """

    # Crear la tabla
    cur = conn.cursor()
    try:
        cur.execute(table_schema)

        # Generar los valores a insertar
        values = [tuple(x) for x in dataframe.values]

        # Definir el INSERT
        insert_sql = f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES %s"
        insert_sql = f"INSERT INTO {table_name} (\"Open\", \"High\", \"Low\", \"Close\", \"Volume\", \"Dividends\", \"Stock Splits\", \"Date\") VALUES %s"


        # Execute the transaction to insert the data
        cur.execute("BEGIN")
        execute_values(cur, insert_sql, values)
        cur.execute("COMMIT")
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()  # Rollback the transaction on error

    print('Proceso terminado')


In [43]:
cargar_en_redshift(conn=conn, table_name='amazon', dataframe=hist)

['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits', 'Date']
Proceso terminado
